In [6]:
# Flag para configurar se plota ou não gráficos e tabelas com o matplotlib.

plot = 0

# Importação de bibliotecas

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import *
from sklearn.preprocessing import LabelEncoder,StandardScaler
import csv



# Pré-Processamento

## Leitura de dados

In [8]:

file_path = 'Data/campeonatos_futebol_atualizacao.csv'

ace_df = pd.read_csv(file_path, encoding='latin-1')

In [9]:
ace_df.describe()

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tratamentos 1,Tratamentos 2,SubstituiÃ§Ãµes 1,SubstituiÃ§Ãµes 2,Tiros-livres 1,Tiros-livres 2,Defesas difÃ­ceis 1,Defesas difÃ­ceis 2,Posse 1(%),Posse 2(%)
count,26204.000000,26204.000000,24942.000000,24942.000000,25388.000000,25388.000000,25392.000000,25392.000000,25394.000000,25394.000000,...,5019.000000,5019.000000,9420.000000,9420.000000,6246.000000,6246.000000,6196.000000,6196.000000,25366.000000,25366.000000
mean,4.767287,3.854526,2.139844,1.968527,5.548409,4.459824,6.034302,4.956758,12.694495,12.946444,...,1.995617,2.134489,3.000318,3.001168,14.156260,14.245757,2.676888,3.206908,51.315659,48.508279
std,2.797705,2.444182,1.754924,1.681298,2.975521,2.620973,3.224260,5.451316,4.337402,4.608542,...,2.258323,2.296141,1.020285,1.033004,4.432963,4.335781,1.842689,2.078474,9.834217,9.808893
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,1.000000,1.000000,3.000000,3.000000,4.000000,3.000000,10.000000,10.000000,...,0.000000,0.000000,3.000000,3.000000,11.000000,11.000000,1.000000,2.000000,45.000000,42.000000
50%,4.000000,4.000000,2.000000,2.000000,5.000000,4.000000,6.000000,5.000000,12.000000,13.000000,...,1.000000,2.000000,3.000000,3.000000,14.000000,14.000000,2.000000,3.000000,52.000000,48.000000
75%,6.000000,5.000000,3.000000,3.000000,7.000000,6.000000,8.000000,7.000000,15.000000,16.000000,...,3.000000,3.000000,3.000000,3.000000,17.000000,17.000000,4.000000,4.000000,58.000000,55.000000
max,90.000000,80.000000,23.000000,15.000000,22.000000,21.000000,23.000000,748.000000,46.000000,180.000000,...,17.000000,16.000000,6.000000,6.000000,36.000000,33.000000,11.000000,17.000000,100.000000,100.000000


In [10]:
ace_df.head()

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tiros-livres 1,Tiros-livres 2,Defesas difÃ­ceis 1,Defesas difÃ­ceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,NaN,NaN,NaN,NaN,77.0,23.0,Glasgow Rangers,Livingston,4-3-3,4-3-3
1,0.0,2.0,0.0,2.0,0.0,4.0,3.0,4.0,19.0,14.0,...,NaN,NaN,NaN,NaN,44.0,56.0,Ross County,St.Johnstone,3-5-2,3-5-2
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,NaN,NaN,NaN,NaN,47.0,53.0,Dundee FC,St. Mirren,4-3-3,3-4-1-2
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,NaN,NaN,NaN,NaN,27.0,73.0,Hearts,Celtic,3-4-3,4-2-3-1
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,NaN,NaN,NaN,NaN,58.0,42.0,Aberdeen,Dundee U.,4-4-2,3-5-1-1


## Criação de colunas adicionais

In [11]:
formation_info = {
    '4-2-3-1': (0.87, 'Equilibrada, com 2 volantes; transições rápidas.'),
    '4-4-2': (0.90, 'Tradicional e sólida; foco no equilíbrio.'),
    '4-3-3': (0.78, 'Agressiva, com 3 atacantes abertos.'),
    '3-5-2': (0.85, 'Meio-campo forte, alas apoiam; defesa com 3 zagueiros.'),
    '4-4-1-1': (0.91, 'Defensiva, com apenas 1 atacante.'),
    '4-3-1-2': (0.88, 'Compacta e com meio centralizado.'),
    '3-4-3': (0.76, 'Bastante ofensiva, poucos defensores fixos.'),
    '4-1-4-1': (0.93, 'Muito segura, com um volante fixo.'),
    '3-4-2-1': (0.82, 'Ofensiva mas com cobertura no meio.'),
    '4-5-1': (0.95, 'Extremamente defensiva, 1 atacante isolado.'),
    '3-4-1-2': (0.84, 'Mais equilibrada, com liberdade no ataque.'),
    '4-1-2-2-1': (0.91, 'Meia ofensiva central + um volante, ainda cautelosa.'),
    '5-3-2': (0.97, 'Linha de 5 defensores, muito fechada.'),
    '4-1-2-1-2': (0.89, 'Meio centralizado com losango; moderadamente ofensiva.'),
    '4-2-2-2': (0.86, 'Meio espaçado, mas flexível.'),
    '4-3-2-1': (0.90, 'Bola no pé e compactação.'),
    '3-5-1-1': (0.89, 'Mais povoado no meio, 1 atacante.'),
    '5-4-1': (0.98, 'Extremamente defensiva.'),
    '4-1-2-3': (0.83, 'Bom equilíbrio, com 3 atacantes.'),
    '4-1-3-2': (0.82, 'Mais presença ofensiva, 2 atacantes.'),
    '3-1-4-2': (0.80, 'Bastante ofensiva com apoio de meio.'),
    '4-2-1-3': (0.79, 'Foco ofensivo com 3 à frente.'),
    '3-2-3-2': (0.78, 'Compacta e ofensiva.'),
    '5-2-3': (0.85, 'Defesa sólida, alas com liberdade.'),
    '5-3-1-1': (0.96, 'Muito defensiva, 1 atacante.'),
    '3-3-3-1': (0.81, 'Equilibrada, meio intenso.'),
    '3-3-2-2': (0.79, 'Bastante ofensiva.'),
    '3-1-3-1-2': (0.77, 'Bastante ofensiva e moderna.'),
    '3-2-4-1': (0.75, 'Muito ofensiva, 4 no meio avançado.')
}


In [12]:
ace_df['Resultado Time 1'] = ace_df.apply(lambda row: '0' if row['Gols 1'] > row['Gols 2'] else '1', axis=1)

In [13]:
# Criação de uma variável juntando a informação de quem joga em casa e quem é visitante.
ace_df['Time 1 x Time 2'] = ace_df['Time 1'].values + ' x '+ace_df['Time 2'].values
# ace_df.drop(['Time 1', 'Time 2'], inplace=  True, axis =1)

## Separação de dados para teste

In [14]:
ace_df_teste = ace_df.sample(frac=0.05,random_state=1)
ace_df.drop(ace_df_teste.index, inplace=True)

## Lidar com Outliers

### Visualizar / Retirar manualmente

In [15]:
analys_flag = 0

In [16]:
# Função com método de remoção 'automático' de outliers, por meio de informações estatísticas
def remove_outliers(df, filter_cols, lowq, supq):
    # Iterate over each numerical column
    for col in filter_cols:
        # Calculate the IQR
        Q1 = df[col].quantile(lowq)
        Q3 = df[col].quantile(supq)
        IQR = Q3 - Q1
        # Define the upper and lower bounds for outliers
        fac = 1.5
        lower_bound = Q1 - fac * IQR
        upper_bound = Q3 + fac * IQR
        print(f"{col} - Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
        # Remove outliers
        #print((df[col] <= upper_bound))
        df = df[( (df[col] <= upper_bound) | (df[col].isna()) ) ] # (df[col] >= lower_bound) & 
    print('\n')
    return df


In [17]:
numeric_columns = ace_df.select_dtypes(include='number').columns

if(analys_flag):
    for column in numeric_columns:
        print("Nome da coluna:", column)
        print("Mínimo:", ace_df[column].min())
        print("Mediana:", ace_df[column].median())
        print("Máximo:", ace_df[column].max())
        print('Razao:', ace_df[column].max()/ace_df[column].median())
        print('\n')



In [18]:
if(analys_flag):

    for column in numeric_columns:
        raz = ace_df[column].max()/ace_df[column].median()
        if (raz > 8):
            print(column)
            print(ace_df[column].max())
            print(ace_df[column].median())
            print('Razao:', ace_df[column].max()/ace_df[column].median())
            print('\n')

In [19]:
# Anotação de outliers : Faltas 2 - ~180 ; Chutes fora 2 748.0 ; Cruzamentos 1/2 ?? : 66, 63 ; 
# Chutes a gol 2 ; Chutes a gol 1 ; Cruzamentos 1 ; cruzamentos 2

In [20]:
if(analys_flag):

    # Calcular os quantis
    # var1, var2, var

    quantil_values = [0.95, 0.99 , 0.999, 0.9998,0.9999]
    quantiles_faltas2 = ace_df['Faltas 2'].quantile(quantil_values)
    quantiles_chutes_fora2 = ace_df['Chutes fora 2'].quantile(quantil_values)
    quantiles_cruzamentos2 = ace_df['Cruzamentos 2'].quantile(quantil_values)

    # Filtrar os valores superiores aos quantis
    selected1 = quantil_values[4]
    selected2 = quantil_values[3]
    selected3 =quantil_values[2]
    superiores_faltas2 = ace_df[ace_df['Faltas 2'] > quantiles_faltas2[selected1]]['Faltas 2']
    superiores_chutes_fora2 = ace_df[ace_df['Chutes fora 2'] > quantiles_chutes_fora2[selected2]]['Chutes fora 2']
    superiores_cruzamentos2 = ace_df[ace_df['Cruzamentos 2'] > quantiles_cruzamentos2[selected3]]['Cruzamentos 2']

    # Exibir os valores superiores aos quantis
    print("Valores superiores de quantil para Faltas 2:")
    print(superiores_faltas2)

    print("Valores superiores de quantil para Chutes fora 2:")
    print(superiores_chutes_fora2)

    print("Valores superiores de quantil para Cruzamentos 2:")
    print(superiores_cruzamentos2)


In [21]:
# Anotação de outliers : Faltas 2 - ~180 ; Chutes fora 2 748.0 ; Cruzamentos 1/2 ?? : 66, 63 ; 
# Chutes a gol 2 ; Chutes a gol 1 ; Cruzamentos 1 ; cruzamentos 2

### Filtragem automática por quantil / IQR

In [22]:
ace_df.columns

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2',
       'Tiros-livres 1', 'Tiros-livres 2', 'Defesas difÃ­ceis 1',
       'Defesas difÃ­ceis 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Position 1', 'Position 2', 'Resultado Time 1', 'Time 1 x Time 2'],
      dtype='object')

In [23]:

filter_cols =  ['Faltas 2', 'Chutes fora 2' , 'Chutes a gol 1', 'Chutes a gol 2'] # numeric_columns # ['Faltas 2']# # ['Faltas 2', 'Chutes fora 2',]
# Apply the remove_outliers function to ace_df

ace_df_cleaned = remove_outliers(ace_df , ['Faltas 2', 'Chutes fora 2'], 0.0, 0.75)
ace_df_cleaned = remove_outliers(ace_df_cleaned , [ 'Chutes a gol 1', 'Chutes a gol 2'], 0.0, 0.999)


index_diff = ace_df.index.difference(ace_df_cleaned.index)

print('Diferença:', len(ace_df) - len(ace_df_cleaned))


Faltas 2 - Lower Bound: -24.0, Upper Bound: 40.0
Chutes fora 2 - Lower Bound: -10.5, Upper Bound: 17.5


Chutes a gol 1 - Lower Bound: -24.0, Upper Bound: 40.0
Chutes a gol 2 - Lower Bound: -21.0, Upper Bound: 35.0


Diferença: 32


In [24]:
print(f"DIferença de linhas: {len(ace_df)} - {len(ace_df_cleaned)} = {len(ace_df) - len(ace_df_cleaned)}")

DIferença de linhas: 26330 - 26298 = 32


In [25]:
ace_df_cleaned

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Defesas difÃ­ceis 1,Defesas difÃ­ceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2,Resultado Time 1,Time 1 x Time 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,NaN,NaN,77.0,23.0,Glasgow Rangers,Livingston,4-3-3,4-3-3,0,Glasgow Rangers x Livingston
1,0.0,2.0,0.0,2.0,0.0,4.0,3.0,4.0,19.0,14.0,...,NaN,NaN,44.0,56.0,Ross County,St.Johnstone,3-5-2,3-5-2,1,Ross County x St.Johnstone
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,NaN,NaN,47.0,53.0,Dundee FC,St. Mirren,4-3-3,3-4-1-2,1,Dundee FC x St. Mirren
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,NaN,NaN,27.0,73.0,Hearts,Celtic,3-4-3,4-2-3-1,0,Hearts x Celtic
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,NaN,NaN,58.0,42.0,Aberdeen,Dundee U.,4-4-2,3-5-1-1,0,Aberdeen x Dundee U.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27711,6.0,2.0,1.0,0.0,9.0,6.0,6.0,5.0,7.0,7.0,...,NaN,NaN,52.0,48.0,Aston Villa,Blackburn,4-4-2,4-5-1,1,Aston Villa x Blackburn
27712,6.0,0.0,4.0,3.0,4.0,2.0,8.0,2.0,11.0,9.0,...,NaN,NaN,52.0,48.0,Arsenal,Fulham,4-2-3-1,4-4-2,0,Arsenal x Fulham
27713,7.0,7.0,2.0,3.0,10.0,2.0,8.0,0.0,10.0,6.0,...,NaN,NaN,55.0,45.0,Manchester Utd,Stoke,4-4-2,4-4-2,0,Manchester Utd x Stoke
27714,6.0,6.0,1.0,2.0,4.0,3.0,4.0,6.0,5.0,3.0,...,NaN,NaN,45.0,55.0,West Ham,Manchester City,4-5-1,4-4-2,1,West Ham x Manchester City


## Lidar com valores nulos

In [26]:
null_map = ace_df_cleaned.isna().sum()

In [27]:
threshold = 0.5 * len(ace_df_cleaned)
for col in numeric_columns:
    # print(f'nulos: {null_map[col]}  ; threshold : {threshold}' )
    if (null_map[col] < threshold):
        ace_df_cleaned[col] = ace_df_cleaned[col].fillna(ace_df_cleaned[col].median(numeric_only= True))

In [28]:
print(f'threshold : {threshold}')

threshold : 13149.0


In [29]:
ace_df_cleaned.isna().sum()

Chutes a gol 1              0
Chutes a gol 2              0
Impedimentos 1              0
Impedimentos 2              0
Escanteios 1                0
Escanteios 2                0
Chutes fora 1               0
Chutes fora 2               0
Faltas 1                    0
Faltas 2                    0
CartÃµes amarelos 1         0
CartÃµes amarelos 2         0
CartÃµes vermelhos 1        0
CartÃµes vermelhos 2        0
Cruzamentos 1           17747
Cruzamentos 2           17747
Laterais 1                  0
Laterais 2                  0
Chutes bloqueados 1     17906
Chutes bloqueados 2     17906
Contra-ataques 1        20357
Contra-ataques 2        20357
Gols 1                      0
Gols 2                      0
Tiro de meta 1          14892
Tiro de meta 2          14892
Tratamentos 1           21549
Tratamentos 2           21549
SubstituiÃ§Ãµes 1       17378
SubstituiÃ§Ãµes 2       17378
Tiros-livres 1          20364
Tiros-livres 2          20364
Defesas difÃ­ceis 1     20411
Defesas di

In [30]:
if (plot):
    # Visualização de histogramas 
    dataset = ace_df_cleaned

    num_cols = len(dataset.select_dtypes(include=['number']).columns)
    num_rows = (num_cols + 2) // 3

    fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(10, 2 * num_rows))
    axes = axes.flatten()

    for i, col in enumerate(dataset.select_dtypes(include=['number']).columns):
        try:
            sns.kdeplot(data=dataset, x=col, ax=axes[i], fill=True)
            axes[i].set_title(f'Density Plot of {col}')
        except Exception as e:
            print(f"Error plotting {col}: {e}")


    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    plt.tight_layout()
    plt.show()


## Codificação de variáveis não-numéricas

In [31]:
select_df = ace_df_cleaned

In [32]:

# Crie uma cópia do dataframe ace_df
ace_df_encoded = select_df.copy()

# Instancie o LabelEncoder
label_encoder_dict = {}

# Itere sobre as colunas do dataframe
for column in ace_df_encoded.columns:
    # Verifique se a coluna é do tipo 'object'
    if (ace_df_encoded[column].dtype == 'object') and (column != 'Position 1')and (column != 'Position 2'):
        label_encoder_dict[column] = LabelEncoder()

        # Aplique a codificação usando o LabelEncoder
        ace_df_encoded[column] = label_encoder_dict[column].fit_transform(ace_df_encoded[column])

# Verifique o resultado
print(ace_df_encoded)

       Chutes a gol 1  Chutes a gol 2  Impedimentos 1  Impedimentos 2  \
0                 8.0             0.0             6.0             3.0   
1                 0.0             2.0             0.0             2.0   
2                 4.0             5.0             1.0             5.0   
3                 4.0             7.0             8.0             1.0   
4                 3.0             1.0             1.0             3.0   
...               ...             ...             ...             ...   
27711             6.0             2.0             1.0             0.0   
27712             6.0             0.0             4.0             3.0   
27713             7.0             7.0             2.0             3.0   
27714             6.0             6.0             1.0             2.0   
27715             4.0             2.0             2.0             2.0   

       Escanteios 1  Escanteios 2  Chutes fora 1  Chutes fora 2  Faltas 1  \
0               7.0           1.0            6

## Criação de colunas adicionais 2

In [33]:
ace_df_encoded['Position 1']

0          4-3-3
1          3-5-2
2          4-3-3
3          3-4-3
4          4-4-2
          ...   
27711      4-4-2
27712    4-2-3-1
27713      4-4-2
27714      4-5-1
27715      4-4-2
Name: Position 1, Length: 26298, dtype: object

In [34]:
# Criação de uma variável que classifica a posição dos jogadores quanto à agressividade. (quanto maior o número, mais defensivo)
ace_df_encoded['Position 1 Score'] = ace_df_encoded['Position 1'].map(lambda x: formation_info.get(x, (None,))[0])
ace_df_encoded['Position 2 Score'] = ace_df_encoded['Position 2'].map(lambda x: formation_info.get(x, (None,))[0])
ace_df_encoded['Position 1 Score'] = ace_df_encoded['Position 1 Score'].fillna(ace_df_encoded['Position 1 Score'].median())
ace_df_encoded['Position 2 Score'] = ace_df_encoded['Position 2 Score'].fillna(ace_df_encoded['Position 1 Score'].median())


In [35]:
pesos_eficiencia = [2,2]
# ace_df_encoded['Defesas difÃ­ceis 2'] = ace_df_encoded['Defesas difÃ­ceis 2'].fillna(0)
# ace_df_encoded['Defesas difÃ­ceis 1'] = ace_df_encoded['Defesas difÃ­ceis 1'].fillna(0)

ace_df_encoded['Eficiencia de ataque 1'] = (
    # Calculo de uma métrica de ataque do time
    ((ace_df_encoded['Chutes a gol 1'] * pesos_eficiencia[0] +
     ace_df_encoded['Escanteios 1'] * pesos_eficiencia[1]
     ) / sum(pesos_eficiencia) ) 
     # Uma posição que prioriza ataque penaliza a métrica.
     * ace_df_encoded['Position 1 Score']
# Essa métrica de ataque é dividida pela média de posse de bola do time, considerando que um time eficiente no ataque 'aproveita' melhor sua posse de bola.
) / (ace_df_encoded['Posse 1(%)']  * 0.02 )
ace_df_encoded['Eficiencia de ataque 2'] = (
    # Calculo de uma métrica de ataque do time
    ((ace_df_encoded['Chutes a gol 2'] * pesos_eficiencia[0] +
     ace_df_encoded['Escanteios 2'] * pesos_eficiencia[1]
     ) / sum(pesos_eficiencia) ) 
     # Uma posição que prioriza ataque penaliza a métrica.
     * ace_df_encoded['Position 2 Score']
# Essa métrica de ataque é dividida pela média de posse de bola do time, considerando que um time eficiente no ataque 'aproveita' melhor sua posse de bola.
) / (ace_df_encoded['Posse 2(%)'] * 0.02)

ace_df_encoded['Eficiencia de ataque 1'] = ace_df_encoded['Eficiencia de ataque 1'].replace([np.inf, -np.inf, np.nan], ace_df_encoded['Eficiencia de ataque 1'].median())
ace_df_encoded['Eficiencia de ataque 2'] = ace_df_encoded['Eficiencia de ataque 2'].replace([np.inf, -np.inf, np.nan], ace_df_encoded['Eficiencia de ataque 1'].median())

In [36]:
# infinito_ou_nan = ace_df_encoded['Eficiencia de ataque 1'].isna().sum() + (np.isinf(ace_df_encoded['Eficiencia de ataque 1'])).sum()

# print(f'Número de valores NaN ou infinitos na coluna de ataque 1: {infinito_ou_nan}')


In [37]:
# ace_df_encoded['Eficiencia de ataque 1'].describe()

## Visualização 

### DF Adicionais

In [38]:
ace_df_cut1 = ace_df[['Resultado Time 1','Gols 1', 'Gols 2']]

In [39]:
ace_df_cut2 = ace_df.dropna()

In [40]:
ace_df.columns

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2',
       'Tiros-livres 1', 'Tiros-livres 2', 'Defesas difÃ­ceis 1',
       'Defesas difÃ­ceis 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Position 1', 'Position 2', 'Resultado Time 1', 'Time 1 x Time 2'],
      dtype='object')

In [41]:
ace_df_cut3 = ace_df.drop(columns = ['Tratamentos 1', 'Tratamentos 2','SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2' ], axis = 1)

### Visualização de 2 variáveis

In [42]:
ace_df.columns

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2',
       'Tiros-livres 1', 'Tiros-livres 2', 'Defesas difÃ­ceis 1',
       'Defesas difÃ­ceis 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Position 1', 'Position 2', 'Resultado Time 1', 'Time 1 x Time 2'],
      dtype='object')

In [43]:
if(plot):
    # Visualização de var1 x var2
    plt.scatter(ace_df['Laterais 2'], ace_df['SubstituiÃ§Ãµes 2'])
    plt.xlabel('Impedimentos 1')
    plt.ylabel('Chutes a gol 1')
    plt.title('Nuvem de pontos - Impedimentos 1 x Chutes a gol 1')
    plt.show()

### Correlação Linear entre variáveis

In [44]:
if (plot):
    corr = ace_df_encoded.corr(numeric_only=True)

    plt.figure(figsize=(36, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title("Correlation Matrix")
    plt.show()


### Visualização de densidade por classe

In [45]:
if (plot):
    import math
    hueVar = 'Resultado Time 1'
    dataset = ace_df_encoded

    # Filtra apenas colunas numéricas, exceto a hueVar
    numeric_cols = dataset.select_dtypes(include='number').columns
    numeric_cols = [col for col in numeric_cols if col != hueVar]

    cols_per_row = 3
    n = len(numeric_cols)
    rows = math.ceil(n / cols_per_row)

    fig, axes = plt.subplots(rows, cols_per_row, figsize=(5*cols_per_row, 4*rows))
    axes = axes.flatten()

    for i, col in enumerate(numeric_cols):
        sns.kdeplot(data=dataset, x=col, hue=hueVar, fill=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col} by {hueVar}')

    # Remove gráficos vazios se o número de colunas não for múltiplo de 3
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()


## Seleção de variáveis

In [46]:
ace_df_encoded.isna().sum()

Chutes a gol 1                0
Chutes a gol 2                0
Impedimentos 1                0
Impedimentos 2                0
Escanteios 1                  0
Escanteios 2                  0
Chutes fora 1                 0
Chutes fora 2                 0
Faltas 1                      0
Faltas 2                      0
CartÃµes amarelos 1           0
CartÃµes amarelos 2           0
CartÃµes vermelhos 1          0
CartÃµes vermelhos 2          0
Cruzamentos 1             17747
Cruzamentos 2             17747
Laterais 1                    0
Laterais 2                    0
Chutes bloqueados 1       17906
Chutes bloqueados 2       17906
Contra-ataques 1          20357
Contra-ataques 2          20357
Gols 1                        0
Gols 2                        0
Tiro de meta 1            14892
Tiro de meta 2            14892
Tratamentos 1             21549
Tratamentos 2             21549
SubstituiÃ§Ãµes 1         17378
SubstituiÃ§Ãµes 2         17378
Tiros-livres 1            20364
Tiros-li

In [47]:
null_columns = ace_df_encoded.columns[ace_df_encoded.isnull().any()].tolist()

null_columns

['Cruzamentos 1',
 'Cruzamentos 2',
 'Chutes bloqueados 1',
 'Chutes bloqueados 2',
 'Contra-ataques 1',
 'Contra-ataques 2',
 'Tiro de meta 1',
 'Tiro de meta 2',
 'Tratamentos 1',
 'Tratamentos 2',
 'SubstituiÃ§Ãµes 1',
 'SubstituiÃ§Ãµes 2',
 'Tiros-livres 1',
 'Tiros-livres 2',
 'Defesas difÃ\xadceis 1',
 'Defesas difÃ\xadceis 2',
 'Position 1',
 'Position 2']

In [48]:
ace_df_encoded.columns

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2',
       'Tiros-livres 1', 'Tiros-livres 2', 'Defesas difÃ­ceis 1',
       'Defesas difÃ­ceis 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Position 1', 'Position 2', 'Resultado Time 1', 'Time 1 x Time 2',
       'Position 1 Score', 'Position 2 Score', 'Eficiencia de ataque 1',
       'Eficiencia de ataque 2'],
      dtype='object')

In [49]:
# # selected_variables = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Faltas 1', 'Faltas 2', 'Laterais 1', 'Posse 1(%)', 'Time 1', 'Time 2', 'Position 1'] + ['Gols 1', 'Gols 2', 'Resultado Time 1']
# selected_variables = ['Chutes a gol 1', 'Escanteios 1', 'CartÃµes amarelos 1',
#        'CartÃµes vermelhos 2', 'Cruzamentos 1', 'Laterais 1',
#        'Contra-ataques 2', 'SubstituiÃ§Ãµes 1', 'SubstituiÃ§Ãµes 2',
#        'Defesas difÃ­ceis 2'] + ['Gols 1', 'Gols 2', 'Resultado Time 1']


# selected_variables2 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
#        'Chutes fora 1', 'Chutes fora 2', 'CartÃµes amarelos 1', 'Laterais 1',
#        'Time 1', 'Time 2']
# selected_variables3 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
#        'Time 1', 'Time 2']
# selected_variables3_2 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
#        'Time 1 x Time 2']

# selected_variables3_3 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
#        'Time 1 x Time 2', 'Eficiencia de ataque 1', 'Eficiencia de ataque 2']
# selected_variables3_4 = ['Chutes a gol 1', 'Chutes a gol 2',
#        'Time 1 x Time 2', 'Eficiencia de ataque 1', 'Eficiencia de ataque 2']


# selected_variables4 = ['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Escanteios 1',
#        'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2', 'CartÃµes amarelos 1',
#        'CartÃµes amarelos 2', 'Laterais 1', 'Laterais 2', 'Time 1', 'Time 2',
#        'Position 1', 'Position 2']


# selected_variables5 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Posse 2(%)',
#        'Eficiencia de ataque 1', 'Eficiencia de ataque 2'] # RFE Pós inserção de eficiência de ataque.


# selected_variables6 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
#        'Chutes fora 1', 'CartÃµes amarelos 1', 'CartÃµes vermelhos 1',
#        'CartÃµes vermelhos 2', 'Defesas difÃ­ceis 1', 'Defesas difÃ­ceis 2'] # Após benchmark de RFE com 1-> 43 variáveis selecionadas

In [50]:

selected_variables7 = ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
       'Chutes fora 1', 'CartÃµes amarelos 1', 'CartÃµes vermelhos 1',
       'CartÃµes vermelhos 2', 'Laterais 1', 'Time 1'] # Após benchmark de RFE com 1-> 43 variáveis selecionadas, 

ace_df_encoded_cut = ace_df_encoded.drop(null_columns,axis = 1) #[selected_variables]

selected_var = selected_variables7

ace_df_encoded_cut = ace_df_encoded_cut[selected_var  + ['Gols 1', 'Gols 2', 'Resultado Time 1']]

#ace_df_encoded_cut = ace_df_encoded

In [51]:
# Plot de duplas de variáveis preditoras, com a variável alvo colorida de acordo com o Label.
if (plot):
    # Gera os pares de variáveis
    pairs = list(combinations(selected_var, 2))

    # Define o número de colunas e calcula o número de linhas necessárias
    n_cols = 3
    n_rows = math.ceil(len(pairs) / n_cols)

    # Cria o grid de subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 5 * n_rows))
    axes = axes.flatten()

    # Plota cada par no grid
    for i, (var1, var2) in enumerate(pairs):
        sns.scatterplot(
            data=ace_df_encoded,
            x=var1,
            y=var2,
            hue='Resultado Time 1',
            ax=axes[i],
            alpha=0.7
        )
        axes[i].set_title(f'{var1} vs {var2}')

    # Remove subplots vazios (caso o número de pares < n_rows * n_cols)
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()


## Rebalanceamento de classes

In [52]:
oversample = 0

In [53]:
ace_df_encoded_cut.groupby('Resultado Time 1').size()

Resultado Time 1
0    11851
1    14447
dtype: int64

### Downsampling

In [54]:
if (not oversample ):
    count_per_class = ace_df_encoded_cut.groupby('Resultado Time 1').size()

    samples_threshold = min(ace_df_encoded_cut.groupby('Resultado Time 1').size())

    for mat, count in count_per_class.items():
        while count > samples_threshold:
            # Pega índices das linhas no grupo 'mat' até atingir o limite
            exceding_samples = ace_df_encoded_cut[ace_df_encoded_cut['Resultado Time 1'] == mat].sample(n=count - samples_threshold).index

            # Faça o drop das linhas excedentes
            ace_df_encoded_cut = ace_df_encoded_cut.drop(exceding_samples)

            # Atualize a contagem para o grupo 'genre'
            count = len(ace_df_encoded_cut[ace_df_encoded_cut['Resultado Time 1'] == mat])

### Resultado


In [55]:
ace_df_encoded_cut.groupby('Resultado Time 1').size()

Resultado Time 1
0    11851
1    11851
dtype: int64

# Modelo de predição

In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from itertools import combinations


## Treinamento e teste, com divisão do dataSet

### Seleção e treinamento

In [62]:
ace_df_encoded_cut.columns

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Escanteios 2',
       'Chutes fora 1', 'CartÃµes amarelos 1', 'CartÃµes vermelhos 1',
       'CartÃµes vermelhos 2', 'Laterais 1', 'Time 1', 'Gols 1', 'Gols 2',
       'Resultado Time 1'],
      dtype='object')

In [63]:

# Split the data into training and testing sets
# X = ace_df_encoded.drop(['Resultado Time 1','Gols 1', 'Gols 2'] + , axis=1)

target_var1 = ['Gols 1']
target_var2 = ['Gols 2']
target_var3 = ['Resultado Time 1']

X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
y = ace_df_encoded_cut[target_var3]

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.15, random_state=42)

if (oversample):
    X_blc, y_blc = SMOTE().fit_resample(X_train, y_train)



In [64]:
if (oversample):
    y_blc.value_counts()

In [65]:
# Create and fit the XGBoost model
# xgb_model = XGBClassifier()
# rf_classifier_model = RandomForestClassifier()

# Create an instance of XGBRegressor

# xgb_reg_model = XGBRegressor()

xgb_classifier_model = XGBClassifier(subsample=1.0, reg_lambda=0.1, reg_alpha=0, n_estimators=50, max_depth=3, learning_rate=0.45, gamma=1, colsample_bytree=0.8)
# extra_trees_model = ExtraTreesClassifier(n_estimators = 700, min_samples_split= 2,min_samples_leaf= 2, max_features ='log2', max_depth= 20, bootstrap = False) # n_estimators = 700, min_samples_split= 2,min_samples_leaf= 2, max_features ='log2', max_depth= 20, bootstrap = False

use_model = xgb_classifier_model# xgb_classifier_model# rf_classifier_model

if (oversample):
    use_model.fit(X_blc, y_blc)
else:
    use_model.fit(X_train, y_train)



### Cross Validation

In [66]:
# Evaluate the model using cross-validation
scores = cross_val_score(use_model, X, y, cv=10)
mean_accuracy = scores.mean()

print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.7030643126242376


In [67]:
scores

array([0.67482075, 0.70097005, 0.72953586, 0.72616034, 0.65949367,
       0.69620253, 0.66793249, 0.7464135 , 0.72827004, 0.70084388])

### Predição de validação

In [70]:
y_pred = use_model.predict(X_validate)

In [71]:
from sklearn.metrics import confusion_matrix
# # Assuming y_test is a pandas Series or DataFrame column
# unique_values = y_test.unique()
# unique_values.sort()

cm = confusion_matrix(y_validate, y_pred)
#cm_df = pd.DataFrame(cm, index=[0,1,2], columns=[0,1,2])
cm

array([[1240,  572],
       [ 477, 1267]])

In [72]:
score = accuracy_score(y_validate, y_pred)
score

0.7050056242969629

### Predição do dataSet de teste inalterado usando o modelo retreinado


In [75]:
ace_df_teste

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Defesas difÃ­ceis 1,Defesas difÃ­ceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2,Resultado Time 1,Time 1 x Time 2
26289,2.0,4.0,3.0,2.0,6.0,7.0,12.0,4.0,9.0,16.0,...,NaN,NaN,54.0,46.0,Sunderland,Aston Villa,4-4-2,4-4-2,1,Sunderland x Aston Villa
6027,3.0,3.0,0.0,4.0,8.0,5.0,4.0,0.0,18.0,13.0,...,1.0,1.0,46.0,54.0,Roma,Inter de MilÃ£o,3-4-3,3-4-1-2,1,Roma x Inter de MilÃ£o
8743,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SSD Parma,Chievo,4-5-1,4-4-2,0,SSD Parma x Chievo
17667,7.0,2.0,2.0,1.0,5.0,2.0,8.0,9.0,14.0,10.0,...,NaN,NaN,60.0,40.0,Le Mans,Boulogne,4-3-3,4-4-2,1,Le Mans x Boulogne
6445,7.0,5.0,1.0,4.0,12.0,3.0,8.0,6.0,17.0,11.0,...,3.0,5.0,51.0,49.0,Cagliari,Udinese,4-3-1-2,3-5-2,1,Cagliari x Udinese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25616,6.0,5.0,4.0,4.0,10.0,6.0,5.0,4.0,5.0,9.0,...,NaN,NaN,47.0,53.0,West Bromwich,Manchester City,4-2-3-1,4-2-3-1,1,West Bromwich x Manchester City
17496,1.0,2.0,1.0,0.0,5.0,5.0,10.0,5.0,7.0,7.0,...,NaN,NaN,55.0,45.0,Auxerre,Saint Etienne,5-4-1,4-2-2-2,1,Auxerre x Saint Etienne
4195,5.0,7.0,3.0,5.0,10.0,5.0,7.0,8.0,10.0,10.0,...,NaN,NaN,52.0,48.0,Start,Aalesund,4-3-3,4-4-2,1,Start x Aalesund
5745,6.0,6.0,0.0,1.0,4.0,11.0,2.0,6.0,7.0,20.0,...,5.0,6.0,37.0,63.0,Lecce,Roma,4-3-1-2,4-2-3-1,1,Lecce x Roma


In [82]:
for col in ace_df_teste.columns:
    if (ace_df_teste[col].dtype == 'object') and (col != 'Position 1') and (col != 'Position 2'):
        #print(col)
        le = label_encoder_dict[col]
        unseen = set(ace_df_teste[col].unique()) - set(le.classes_)
        if unseen:
            le.classes_ = np.append(le.classes_, list(unseen))
        ace_df_teste[col] = le.transform(ace_df_teste[col])

ace_df_teste['Position 1 Score'] = ace_df_teste['Position 1'].map(lambda x: formation_info.get(x, (None,))[0])
ace_df_teste['Position 2 Score'] = ace_df_teste['Position 2'].map(lambda x: formation_info.get(x, (None,))[0])


In [83]:
X_test = ace_df_teste.drop(target_var1+target_var2+target_var3, axis=1)[selected_var]
y_test = ace_df_teste[target_var3[0]].astype(int)


In [84]:
X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
y = ace_df_encoded_cut[target_var3]

xgb_classifier_model = XGBClassifier(subsample=1.0, reg_lambda=0.1, reg_alpha=0, n_estimators=50, max_depth=3, learning_rate=0.45, gamma=1, colsample_bytree=0.8)

xgb_classifier_model.fit(X, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.45, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, ...)

In [85]:
y_teste_pred = xgb_classifier_model.predict(X_test)

In [86]:
cm = confusion_matrix(y_test, y_teste_pred)
cm

array([[314, 317],
       [143, 612]])

In [87]:
# Acurácia do modelo com dados não vistos / modificados.

score = accuracy_score(y_test, y_teste_pred)
score

0.6681096681096681

## Benchmark


In [ ]:
# Flag para indicar se é desejável realizar os benchmark's
bench = 0

### RFE - n features

In [543]:

# Split the data into training and testing sets
# X = ace_df_encoded.drop(['Resultado Time 1','Gols 1', 'Gols 2'] + , axis=1)

target_var1 = ['Gols 1']
target_var2 = ['Gols 2']
target_var3 = ['Resultado Time 1']

X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
y = ace_df_encoded_cut[target_var3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)



In [544]:
X

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Defesas difÃ­ceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Time 1 x Time 2,Position 1 Score,Position 2 Score,Eficiencia de ataque 1,Eficiencia de ataque 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,0.0,77.0,23.0,118,170,2760,0.78,0.78,0.044740,0.008478
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,0.0,47.0,53.0,85,258,1955,0.78,0.84,0.055319,0.070000
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,0.0,27.0,73.0,128,68,2941,0.76,0.87,0.079753,0.042705
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,0.0,58.0,42.0,3,86,56,0.90,0.89,0.034914,0.028254
5,5.0,5.0,1.0,0.0,2.0,4.0,3.0,7.0,21.0,11.0,...,0.0,43.0,57.0,190,133,4479,0.87,0.87,0.043837,0.040702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27711,6.0,2.0,1.0,0.0,9.0,6.0,6.0,5.0,7.0,7.0,...,0.0,52.0,48.0,22,41,477,0.90,0.95,0.073558,0.042882
27712,6.0,0.0,4.0,3.0,4.0,2.0,8.0,2.0,11.0,9.0,...,0.0,52.0,48.0,20,107,432,0.87,0.90,0.050192,0.009375
27713,7.0,7.0,2.0,3.0,10.0,2.0,8.0,0.0,10.0,6.0,...,0.0,55.0,45.0,179,262,4213,0.90,0.90,0.079091,0.056667
27714,6.0,6.0,1.0,2.0,4.0,3.0,4.0,6.0,5.0,3.0,...,0.0,45.0,55.0,298,178,7100,0.95,0.90,0.063333,0.045000


In [545]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Definir o estimador
estimator = use_model

# Criar o objeto RFE
rfe = RFE(estimator, n_features_to_select=10)

# Ajustar a RFE aos dados
rfe.fit(X, y)

# Obter os rankings das features
feature_ranking = rfe.ranking_

# Obter as features selecionadas
selected_features = X.columns[rfe.support_]

In [533]:
selected_features

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Posse 2(%)',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object')

In [ ]:
# Seleção com 10 features

# ['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1', 'Faltas 1',
#        'Faltas 2', 'Laterais 1', 'Posse 1(%)', 'Time 1', 'Time 2',
#        'Position 1'],
#       dtype='object'

### RFE - Best n features

In [718]:
def downsampling(ace_df_encoded_cut):
    ace_df_encoded_cut2 = ace_df_encoded_cut.copy()
    count_per_class = ace_df_encoded_cut2.groupby('Resultado Time 1').size()

    samples_threshold = min(ace_df_encoded_cut2.groupby('Resultado Time 1').size())

    for mat, count in count_per_class.items():
        while count > samples_threshold:
            # Pega índices das linhas no grupo 'mat' até atingir o limite
            exceding_samples = ace_df_encoded_cut2[ace_df_encoded_cut2['Resultado Time 1'] == mat].sample(n=count - samples_threshold).index

            # Faça o drop das linhas excedentes
            ace_df_encoded_cut2 = ace_df_encoded_cut2.drop(exceding_samples)

            # Atualize a contagem para o grupo 'genre'
            count = len(ace_df_encoded_cut2[ace_df_encoded_cut2['Resultado Time 1'] == mat])
    return ace_df_encoded_cut2

In [719]:
print(f'Há {len(ace_df_encoded.columns) - 3} features que podem ser selecionadas.')

Há 43 features que podem ser selecionadas.


In [720]:
best_features_selection = [-1,0,[],[]]
downsamp = 1
for n_features in [i for i in range(1, 43)]:
    ace_df_encoded_cut = ace_df_encoded.drop(null_columns,axis = 1) #[selected_variables]
    
    ace_df_encoded_cut = downsampling(ace_df_encoded_cut) if downsamp else ace_df_encoded_cut

    target_var1 = ['Gols 1']
    target_var2 = ['Gols 2']
    target_var3 = ['Resultado Time 1']

    X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
    y = ace_df_encoded_cut[target_var3]

    # Definir o estimador
    xgb_classifier_model = XGBClassifier(subsample=1.0, reg_lambda=0.1, reg_alpha=0, n_estimators=50, max_depth=3, learning_rate=0.45, gamma=1, colsample_bytree=0.8)

    rfe = RFE(xgb_classifier_model, n_features_to_select= n_features)

    # Ajustar a RFE aos dados
    rfe.fit(X, y)

    # Obter os rankings das features
    feature_ranking = rfe.ranking_

    # Obter as features selecionadas
    selected_features = X.columns[rfe.support_]

    ace_df_encoded_cut = ace_df_encoded.drop(null_columns,axis = 1) #[selected_variables]

    ace_df_encoded_cut = ace_df_encoded_cut[list(selected_features)  + ['Gols 1', 'Gols 2', 'Resultado Time 1']]

    ace_df_encoded_cut = downsampling(ace_df_encoded_cut) if downsamp else ace_df_encoded_cut


    X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
    y = ace_df_encoded_cut[target_var3]

    # Evaluate the model using cross-validation
    scores = cross_val_score(use_model, X, y, cv=10)
    mean_accuracy = scores.mean()
    if (mean_accuracy > best_features_selection[1]):
        best_features_selection = [n_features, mean_accuracy, selected_features, feature_ranking]
    print(f" num de features : {n_features} ; Best CV Mean Accuracy: {mean_accuracy} ; selected features: {selected_features} ; ranking: {feature_ranking}")
    
        

 num de features : 1 ; Best CV Mean Accuracy: 0.6320104981171477 ; selected features: Index(['Chutes a gol 1'], dtype='object') ; ranking: [ 1  2 11 25  3  7  8 12 24 21  6 18  5  4  9 15 23 17 22 19 10 13 16 14
 20]
 num de features : 2 ; Best CV Mean Accuracy: 0.6827725470812316 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2'], dtype='object') ; ranking: [ 1  1 14 22  2  6  7  9 23 12  5 18  4  3  8 10 24 19 15 16 11 13 17 21
 20]
 num de features : 3 ; Best CV Mean Accuracy: 0.6853132313794152 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'CartÃµes vermelhos 1'], dtype='object') ; ranking: [ 1  1 14 23  2  5  6 10 20 15  4 19  1  3  7  8 16 18  9 11 17 12 13 22
 21]
 num de features : 4 ; Best CV Mean Accuracy: 0.6962227075752793 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Escanteios 1',
       'CartÃµes vermelhos 1'],
      dtype='object') ; ranking: [ 1  1 12 17  1  4  5  9 19 15  3 22  1  2  6  7 21 13 16 10 14 11  8 20
 18]
 

/home/abraaopc/Documents/projects/PS-2025.1_tango/venv1/lib/python3.10/site-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=26 > n_features=25. There will be no feature selection and all features will be kept.
  warnings.warn(


 num de features : 26 ; Best CV Mean Accuracy: 0.7002086571501736 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Laterais 1',
       'Laterais 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Time 1 x Time 2', 'Position 1 Score', 'Position 2 Score',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object') ; ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/home/abraaopc/Documents/projects/PS-2025.1_tango/venv1/lib/python3.10/site-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=27 > n_features=25. There will be no feature selection and all features will be kept.
  warnings.warn(


 num de features : 27 ; Best CV Mean Accuracy: 0.705339057323273 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Laterais 1',
       'Laterais 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Time 1 x Time 2', 'Position 1 Score', 'Position 2 Score',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object') ; ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


...

 num de features : 40 ; Best CV Mean Accuracy: 0.6991627411629019 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Laterais 1',
       'Laterais 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Time 1 x Time 2', 'Position 1 Score', 'Position 2 Score',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object') ; ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/home/abraaopc/Documents/projects/PS-2025.1_tango/venv1/lib/python3.10/site-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=41 > n_features=25. There will be no feature selection and all features will be kept.
  warnings.warn(


 num de features : 41 ; Best CV Mean Accuracy: 0.7010058669159933 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Laterais 1',
       'Laterais 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Time 1 x Time 2', 'Position 1 Score', 'Position 2 Score',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object') ; ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/home/abraaopc/Documents/projects/PS-2025.1_tango/venv1/lib/python3.10/site-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=42 > n_features=25. There will be no feature selection and all features will be kept.
  warnings.warn(


 num de features : 42 ; Best CV Mean Accuracy: 0.6983649606854099 ; selected features: Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'CartÃµes amarelos 1', 'CartÃµes amarelos 2',
       'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2', 'Laterais 1',
       'Laterais 2', 'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2',
       'Time 1 x Time 2', 'Position 1 Score', 'Position 2 Score',
       'Eficiencia de ataque 1', 'Eficiencia de ataque 2'],
      dtype='object') ; ranking: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [722]:
best_features_selection

[19,
 np.float64(0.7055892771713346),
 Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Escanteios 1',
        'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2', 'Faltas 2',
        'CartÃµes amarelos 1', 'CartÃµes vermelhos 1', 'CartÃµes vermelhos 2',
        'Laterais 1', 'Laterais 2', 'Posse 1(%)', 'Time 1', 'Time 1 x Time 2',
        'Position 1 Score', 'Position 2 Score', 'Eficiencia de ataque 1'],
       dtype='object'),
 array([1, 1, 1, 7, 1, 1, 1, 1, 6, 1, 1, 5, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1,
        1, 1, 3])]

### Tunning

In [485]:

# Split the data into training and testing sets
# X = ace_df_encoded.drop(['Resultado Time 1','Gols 1', 'Gols 2'] + , axis=1)

target_var1 = ['Gols 1']
target_var2 = ['Gols 2']
target_var3 = ['Resultado Time 1']

X = ace_df_encoded_cut.drop(target_var1+target_var2+target_var3, axis=1)
y = ace_df_encoded_cut[target_var3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)



In [498]:
models = {
    'XGBoost': {
        'model': XGBClassifier( eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200, 300],
            'max_depth': [3, 5, 7, 10],
            'learning_rate': [0.001,0.01, 0.05, 0.1, 0.3],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'gamma': [0, 0.1, 0.3, 1],
            'reg_alpha': [0, 0.01, 0.1],
            'reg_lambda': [0.1, 1, 10],
        }
    },
    # 'Extra Trees': {
    #     'model': ExtraTreesClassifier(),
    #     'params': {
    #         'n_estimators': [250, 350, 500,700],
    #         'max_depth': [None, 10, 20, 30, 60, 100],
    #         'min_samples_split': [2, 5, 10],
    #         'min_samples_leaf': [1, 2, 4, 8 , 12],
    #         'max_features': ['sqrt', 'log2', None],
    #         'bootstrap': [True, False]
    #     }
    # }
}


In [499]:
for name, mp in models.items():
    print(f"🔍 Treinando {name}...")
    clf = RandomizedSearchCV(
        mp['model'],
        mp['params'],
        n_iter=20,
        scoring='accuracy',
        cv=5,
        random_state=42,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)
    
    best_model = clf.best_estimator_
    test_acc = cross_val_score(best_model, X, y, cv=5).mean()
    
    print(f"✅ {name} - Test Accuracy: {test_acc:.4f}")
    print(f"Best Params: {clf.best_params_}\n")


🔍 Treinando XGBoost...


/home/abraaopc/Documents/projects/PS-2025.1_tango/venv1/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


✅ XGBoost - Test Accuracy: 0.7045
Best Params: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 0, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.3, 'gamma': 1, 'colsample_bytree': 0.8}



In [137]:
test_acc

np.float64(0.6933491483162798)